In [ ]:
# Установка transformers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Установка catboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from itertools import groupby

In [ ]:
#import необходимых модулей

import pandas as pd
import yaml
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

import numpy as np
from datetime import datetime, timedelta

from catboost import CatBoostClassifier, Pool

In [ ]:
#Считывание данных в DataFrame 
train= pd.read_csv('train_predobr_for22.csv', sep=';', index_col=None)
test= pd.read_csv('test_22.csv', sep=';', index_col=None)
tabliza= pd.read_csv('tabliza_clasters_for22.csv', sep=';', index_col=None)
control_test=pd.read_csv('control_test.csv', sep=';', index_col=None)

In [ ]:
train

,Номер кластера исходный,claster_centr_ishodn,Расстояние до центра кластера,overall_worklogs,project_id,Id_razn,mean_words_comment,count_comments,id,assignee_id,creator_id,summary_to_eng,Year,Month,Day,Hour,DayOfWeek,Mesto_in_project,Text_Summa
0,0,16897,0.017107,14400,5,0,0.0,0,819947,93,93,"UI tests of the section ""Personal account""",2019,10,1,6,2,0.038462,"UI tests of the section ""Personal account"""
1,0,16897,0.062319,7800,5,475,16.0,2,819933,207,1,Make a division by mobile operators,2019,10,1,6,2,0.153846,Make a division by mobile operators Added the ...
2,0,16897,0.004677,17580,5,17,30.0,2,819924,264,207,Doesn't work with cdr_calc_durations,2019,10,1,8,2,0.230769,Doesn't work with cdr_calc_durations [~mursidi...
3,0,16897,0.038380,22500,5,0,0.0,0,819921,94,94,Fix crash addons (NegativeArraySizeException),2019,10,1,11,2,0.250000,Fix crash addons (NegativeArraySizeException)
4,0,16897,0.054099,9000,5,0,0.0,0,819910,1,1,Checking the client's balance when changing th...,2019,10,2,13,3,0.326923,Checking the client's balance when changing th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268,0,16897,0.034370,11880,48,0,0.0,0,688457,1,1,Investigate ANTMAN and find working CODs,2020,6,12,18,5,5.113941,Investigate ANTMAN and find working CODs
4269,6,61906,0.035446,67080,48,0,62.0,1,688422,1,1,Fix update script bug with JackfruitUsers,2020,6,12,20,5,5.194370,Fix update script bug with JackfruitUsers Comm...
4270,0,16897,0.017107,14400,48,0,11.0,1,688417,1,116,Teachers page returns 500 error,2020,6,15,9,1,5.207775,Teachers page returns 500 error fixed. This is...
4271,0,16897,0.062630,26040,48,0,0.0,0,688414,1,116,Make a test to check teachers pages for VN & TH,2020,6,15,9,1,5.214477,Make a test to check teachers pages for VN & TH


In [ ]:
test

,id,Id_razn,mean_words_comment,count_comments,project_id,Mesto_in_project,Year,Month,Day,Hour,DayOfWeek,assignee_id,creator_id,summary_to_eng,Text_Summa
0,675975,478,62.000000,8,5,8.884615,2020,1,15,9,3,112,112,Download all mp3 files of calls to disk,Download all mp3 files of calls to disk Resear...
1,675972,721,16.500000,4,5,8.903846,2020,1,15,9,3,112,112,Delete mp3 call files from the server before 0...,Delete mp3 call files from the server before 0...
2,675965,0,7.000000,1,5,8.961538,2020,1,16,4,4,94,54,Remove the tariff selection window and transfe...,Remove the tariff selection window and transfe...
3,675949,0,0.000000,0,5,9.153846,2020,1,17,8,5,54,54,Clear the project backlog of irrelevant tasks,Clear the project backlog of irrelevant tasks
4,675938,13,45.500000,2,5,9.269231,2020,1,20,4,1,94,54,Attach Firebase and GitLab bots to NambaTaxi S...,Attach Firebase and GitLab bots to NambaTaxi S...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,678981,20,34.833333,6,48,1.340483,2020,6,29,15,1,1,1,Logic of countdown clock in Payment Page,Logic of countdown clock in Payment Page [~sar...
485,688228,8,35.333333,3,48,5.623324,2020,6,30,15,2,82,82,Restoration of the monitoring and log system,Restoration of the monitoring and log system P...
486,688157,0,0.000000,0,48,5.757373,2020,7,8,0,3,82,82,Deploy EVN and ETL,Deploy EVN and ETL
487,688147,0,41.000000,1,48,5.770777,2020,7,9,3,4,82,82,Prepare documentation for Magento,Prepare documentation for Magento * Prepare in...


In [ ]:
tabliza

,claster_centr,Номер кластера
0,16897,0
1,7014000,1
2,3393960,2
3,1162850,3
4,1955910,4
5,484513,5
6,61906,6
7,182284,7


In [ ]:
train.columns

Index(['Номер кластера исходный', 'claster_centr_ishodn',
       'Расстояние до центра кластера', 'overall_worklogs', 'project_id',
       'Id_razn', 'mean_words_comment', 'count_comments', 'id', 'assignee_id',
       'creator_id', 'summary_to_eng', 'Year', 'Month', 'Day', 'Hour',
       'DayOfWeek', 'Mesto_in_project', 'Text_Summa'],
      dtype='object')

In [ ]:
# выбираем поля для нейросети СatBoost по бинарной классификации

X_train=train[['project_id', 'assignee_id', 'creator_id', 'summary_to_eng',
        'Year', 'Month', 'Day', 'Hour', 'DayOfWeek', 'Mesto_in_project',
       'Id_razn', 'mean_words_comment', 'count_comments', 'Text_Summa' ]]
y_train=train[['Номер кластера исходный']]


testnet=test[['project_id', 'assignee_id', 'creator_id', 'summary_to_eng',
        'Year', 'Month', 'Day', 'Hour', 'DayOfWeek','Mesto_in_project',
       'Id_razn', 'mean_words_comment', 'count_comments','Text_Summa' ]]

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4273 entries, 0 to 4272
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   project_id          4273 non-null   int64  
 1   assignee_id         4273 non-null   int64  
 2   creator_id          4273 non-null   int64  
 3   summary_to_eng      4273 non-null   object 
 4   Year                4273 non-null   int64  
 5   Month               4273 non-null   int64  
 6   Day                 4273 non-null   int64  
 7   Hour                4273 non-null   int64  
 8   DayOfWeek           4273 non-null   int64  
 9   Mesto_in_project    4273 non-null   float64
 10  Id_razn             4273 non-null   int64  
 11  mean_words_comment  4273 non-null   float64
 12  count_comments      4273 non-null   int64  
 13  Text_Summa          4273 non-null   object 
dtypes: float64(2), int64(10), object(2)
memory usage: 467.5+ KB


In [ ]:
# Определяем веса для классов кластеров
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes = np.unique(y_train), y = np.ravel(y_train))
class_weights = dict(zip(classes, weights))
class_weights

{0: 0.14903041294642858,
 1: 534.125,
 2: 534.125,
 3: 89.02083333333333,
 4: 133.53125,
 5: 28.111842105263158,
 6: 0.9387082601054482,
 7: 6.001404494382022}

In [ ]:
#Обучаем 

pool_train_solution = Pool(X_train, y_train, cat_features = ['project_id', 'assignee_id', 'creator_id','Year', 'Month', 'Day', 'Hour', 'DayOfWeek'], text_features = ['summary_to_eng', 'Text_Summa'])
pool_test_solution = Pool(testnet,cat_features = ['project_id', 'assignee_id', 'creator_id','Year', 'Month', 'Day', 'Hour', 'DayOfWeek'], text_features = ['summary_to_eng', 'Text_Summa'])

model_test = CatBoostClassifier(iterations=100,  task_type='GPU',depth=12, verbose =True, random_seed=24,# learning_rate=0.015,
                               leaf_estimation_method= 'Gradient',bootstrap_type= 'Poisson',objective = 'MultiClass',eval_metric = 'Accuracy',loss_function='MultiClassOneVsAll', class_weights=class_weights)
model_test.fit(pool_train_solution)

0:	learn: 0.6403968	total: 373ms	remaining: 6m 12s
1:	learn: 0.7310175	total: 635ms	remaining: 5m 16s
2:	learn: 0.7947960	total: 928ms	remaining: 5m 8s
3:	learn: 0.8151873	total: 1.24s	remaining: 5m 8s
4:	learn: 0.8229131	total: 1.56s	remaining: 5m 10s
5:	learn: 0.8204930	total: 1.85s	remaining: 5m 5s
6:	learn: 0.8440034	total: 2.14s	remaining: 5m 3s
7:	learn: 0.8441263	total: 2.42s	remaining: 4m 59s
8:	learn: 0.8458587	total: 2.69s	remaining: 4m 56s
9:	learn: 0.8462955	total: 3s	remaining: 4m 56s
10:	learn: 0.8470488	total: 3.27s	remaining: 4m 54s
11:	learn: 0.8480670	total: 3.55s	remaining: 4m 52s
12:	learn: 0.8501238	total: 3.83s	remaining: 4m 50s
13:	learn: 0.8402975	total: 4.09s	remaining: 4m 48s
14:	learn: 0.8422850	total: 4.36s	remaining: 4m 46s
15:	learn: 0.8490967	total: 4.65s	remaining: 4m 45s
16:	learn: 0.8498874	total: 4.92s	remaining: 4m 44s
17:	learn: 0.8589484	total: 5.21s	remaining: 4m 44s
18:	learn: 0.8599667	total: 5.5s	remaining: 4m 43s
19:	learn: 0.8659902	total: 5.

In [ ]:
# предварительный тест схождения на всем обучающем множестве
y_pred = model_test.predict(X_train)
y_pred

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [ ]:
print("Значение метрики f1 на трейне по кластерам: ", f1_score(train['Номер кластера исходный'], y_pred, average='macro', zero_division = 0))

Значение метрики f1 на трейне по кластерам:  0.7286893029984437


In [ ]:
train

,Номер кластера исходный,claster_centr_ishodn,Расстояние до центра кластера,overall_worklogs,project_id,Id_razn,mean_words_comment,count_comments,id,assignee_id,creator_id,summary_to_eng,Year,Month,Day,Hour,DayOfWeek,Mesto_in_project,Text_Summa
0,0,16897,0.017107,14400,5,0,0.0,0,819947,93,93,"UI tests of the section ""Personal account""",2019,10,1,6,2,0.038462,"UI tests of the section ""Personal account"""
1,0,16897,0.062319,7800,5,475,16.0,2,819933,207,1,Make a division by mobile operators,2019,10,1,6,2,0.153846,Make a division by mobile operators Added the ...
2,0,16897,0.004677,17580,5,17,30.0,2,819924,264,207,Doesn't work with cdr_calc_durations,2019,10,1,8,2,0.230769,Doesn't work with cdr_calc_durations [~mursidi...
3,0,16897,0.038380,22500,5,0,0.0,0,819921,94,94,Fix crash addons (NegativeArraySizeException),2019,10,1,11,2,0.250000,Fix crash addons (NegativeArraySizeException)
4,0,16897,0.054099,9000,5,0,0.0,0,819910,1,1,Checking the client's balance when changing th...,2019,10,2,13,3,0.326923,Checking the client's balance when changing th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268,0,16897,0.034370,11880,48,0,0.0,0,688457,1,1,Investigate ANTMAN and find working CODs,2020,6,12,18,5,5.113941,Investigate ANTMAN and find working CODs
4269,6,61906,0.035446,67080,48,0,62.0,1,688422,1,1,Fix update script bug with JackfruitUsers,2020,6,12,20,5,5.194370,Fix update script bug with JackfruitUsers Comm...
4270,0,16897,0.017107,14400,48,0,11.0,1,688417,1,116,Teachers page returns 500 error,2020,6,15,9,1,5.207775,Teachers page returns 500 error fixed. This is...
4271,0,16897,0.062630,26040,48,0,0.0,0,688414,1,116,Make a test to check teachers pages for VN & TH,2020,6,15,9,1,5.214477,Make a test to check teachers pages for VN & TH


In [ ]:
# Проверим уровень схождения по r2 на трейне
train['Номер кластера']=y_pred
train=train.merge(tabliza, how="left", on="Номер кластера")
train['Номер кластера'].value_counts()

0    3831
6     367
7      57
5      12
3       3
2       1
1       1
4       1
Name: Номер кластера, dtype: int64

In [ ]:
train['Номер кластера исходный'].value_counts()

0    3584
6     569
7      89
5      19
3       6
4       4
2       1
1       1
Name: Номер кластера исходный, dtype: int64

In [ ]:
train

,Номер кластера исходный,claster_centr_ishodn,Расстояние до центра кластера,overall_worklogs,project_id,Id_razn,mean_words_comment,count_comments,id,assignee_id,...,summary_to_eng,Year,Month,Day,Hour,DayOfWeek,Mesto_in_project,Text_Summa,Номер кластера,claster_centr
0,0,16897,0.017107,14400,5,0,0.0,0,819947,93,...,"UI tests of the section ""Personal account""",2019,10,1,6,2,0.038462,"UI tests of the section ""Personal account""",0,16897
1,0,16897,0.062319,7800,5,475,16.0,2,819933,207,...,Make a division by mobile operators,2019,10,1,6,2,0.153846,Make a division by mobile operators Added the ...,0,16897
2,0,16897,0.004677,17580,5,17,30.0,2,819924,264,...,Doesn't work with cdr_calc_durations,2019,10,1,8,2,0.230769,Doesn't work with cdr_calc_durations [~mursidi...,0,16897
3,0,16897,0.038380,22500,5,0,0.0,0,819921,94,...,Fix crash addons (NegativeArraySizeException),2019,10,1,11,2,0.250000,Fix crash addons (NegativeArraySizeException),0,16897
4,0,16897,0.054099,9000,5,0,0.0,0,819910,1,...,Checking the client's balance when changing th...,2019,10,2,13,3,0.326923,Checking the client's balance when changing th...,0,16897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268,0,16897,0.034370,11880,48,0,0.0,0,688457,1,...,Investigate ANTMAN and find working CODs,2020,6,12,18,5,5.113941,Investigate ANTMAN and find working CODs,0,16897
4269,6,61906,0.035446,67080,48,0,62.0,1,688422,1,...,Fix update script bug with JackfruitUsers,2020,6,12,20,5,5.194370,Fix update script bug with JackfruitUsers Comm...,0,16897
4270,0,16897,0.017107,14400,48,0,11.0,1,688417,1,...,Teachers page returns 500 error,2020,6,15,9,1,5.207775,Teachers page returns 500 error fixed. This is...,0,16897
4271,0,16897,0.062630,26040,48,0,0.0,0,688414,1,...,Make a test to check teachers pages for VN & TH,2020,6,15,9,1,5.214477,Make a test to check teachers pages for VN & TH,0,16897


In [ ]:
print("Значение метрики r2 на трейне по секундам: ", r2_score(train['claster_centr'], train['overall_worklogs']))

Значение метрики r2 на трейне по секундам:  0.6968838610098933


In [ ]:
#Получение ответов
y_pred_solution = model_test.predict(pool_test_solution)

In [ ]:
#Контрольные ответы по 3-м наибольшим известным
control_solution = model_test.predict(control_test)
control_solution

In [ ]:
# Заводим результаты в тест
test['Номер кластера']=y_pred_solution
test['Номер кластера'].value_counts()

0    469
6     16
5      2
7      2
Name: Номер кластера, dtype: int64

In [ ]:
# заводим центры кластеров в качестве ответа
test=test.merge(tabliza, how="left", on="Номер кластера")
test

,id,Id_razn,mean_words_comment,count_comments,project_id,Mesto_in_project,Year,Month,Day,Hour,DayOfWeek,assignee_id,creator_id,summary_to_eng,Text_Summa,Номер кластера,claster_centr
0,675975,478,62.000000,8,5,8.884615,2020,1,15,9,3,112,112,Download all mp3 files of calls to disk,Download all mp3 files of calls to disk Resear...,0,16897
1,675972,721,16.500000,4,5,8.903846,2020,1,15,9,3,112,112,Delete mp3 call files from the server before 0...,Delete mp3 call files from the server before 0...,0,16897
2,675965,0,7.000000,1,5,8.961538,2020,1,16,4,4,94,54,Remove the tariff selection window and transfe...,Remove the tariff selection window and transfe...,0,16897
3,675949,0,0.000000,0,5,9.153846,2020,1,17,8,5,54,54,Clear the project backlog of irrelevant tasks,Clear the project backlog of irrelevant tasks,0,16897
4,675938,13,45.500000,2,5,9.269231,2020,1,20,4,1,94,54,Attach Firebase and GitLab bots to NambaTaxi S...,Attach Firebase and GitLab bots to NambaTaxi S...,0,16897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,678981,20,34.833333,6,48,1.340483,2020,6,29,15,1,1,1,Logic of countdown clock in Payment Page,Logic of countdown clock in Payment Page [~sar...,6,61906
485,688228,8,35.333333,3,48,5.623324,2020,6,30,15,2,82,82,Restoration of the monitoring and log system,Restoration of the monitoring and log system P...,0,16897
486,688157,0,0.000000,0,48,5.757373,2020,7,8,0,3,82,82,Deploy EVN and ETL,Deploy EVN and ETL,0,16897
487,688147,0,41.000000,1,48,5.770777,2020,7,9,3,4,82,82,Prepare documentation for Magento,Prepare documentation for Magento * Prepare in...,0,16897


In [ ]:
test_itog=test[['id','claster_centr']]
test_itog.columns=[['id','overall_worklogs']]
test_itog

,id,overall_worklogs
0,675975,16897
1,675972,16897
2,675965,16897
3,675949,16897
4,675938,16897
...,...,...
484,678981,61906
485,688228,16897
486,688157,16897
487,688147,16897


In [ ]:
test_itog.to_csv('прогноз итоговый по 22 нейзвестным сид 24.csv', sep=',', index=None)